### with multithreading 

In [1]:
import requests 
import concurrent.futures 
import csv
import os 

api_url = 'https://www.alphavantage.co/query?'
api_key =os.environ['ALPHA_ADVANTAGE']
params = {'function':'TIME_SERIES_DAILY_ADJUSTED',
              'outputsize':'full',
              'datatype':'csv',
             'apikey': api_key}

with open("./nasdaqlisted.txt", 'r') as f:
    stock_listed = f.read()
symbols = [line.split('|')[0] for line in stock_listed.split("\n")[1:10]]

In [2]:
def download_data_full(symbol):
	'''
	download the full-length time series of 
	up to 20 years of historical data for a symbol
	'''
	params['symbol'] = symbol 

	response = requests.get(api_url, params=params)
	if response.status_code == 200:
	    data = response.content.decode('utf-8')
	else:
	    print('[!] HTTP {0} calling [{1}]'.format(response.status_code, api_url))

	result = [data.split(',') for data in data.split('\r\n')]
	return symbol, result 

In [6]:
%%timeit
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    result = executor.map(download_data_full, symbols[:10])
result = list(result)

The slowest run took 6.10 times longer than the fastest. This could mean that an intermediate result is being cached.
13.8 s ± 4.57 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


### without mutlithreading

In [12]:
%%timeit
for symbol in symbols:
        try:
            params['symbol'] = symbol 
            response = requests.get(api_url, params=params)
            if response.status_code == 200:
                data = response.content.decode('utf-8')
            else:
                print('[!] HTTP {0} calling [{1}]'.format(response.status_code, api_url))
            result = [data.split(',') for data in data.split('\r\n')]
        except Exception as e:
            print(e)

52.5 s ± 1.21 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
